# Keras
## Sequential model

Sequential model is appropriate for plain stack of layers. Each layer
has exactly one input tensor and one output tensor.


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-05-15 04:42:51.866123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = keras.Sequential([
    layers.Dense(2, activation='relu', name='layer1'),
    layers.Dense(3, activation='relu', name='layer2'),
    layers.Dense(4, activation='relu', name='layer3')
])

x = tf.ones((3,3))
y = model(x)

is equivalent to

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

A Sequential model is not appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

## Create Sequential model
Pass list of layer to Sequatial constructor

In [4]:
model = keras.Sequential([
    layers.Dense(2, activation='relu'),
    layers.Dense(3, activation='relu'),
    layers.Dense(4)
])

In [5]:
model.layers

Create incrementally with add function

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

Remove layer with 'pop' function

In [6]:
model.pop()
print(len(model.layers))

2


Name argument is useful in tensorboard.

## Input Shape
Generally, all layers in Keras need to know the shape of their inputs in order to be able to create their weights. So when you create a layer like this, initially, it has no weights

In [7]:
layer = layers.Dense(3)
layer.weights # empty

[]

In [10]:
x = tf.ones((1,4))
y = layer(x)
layer.weights

[<tf.Variable 'dense_3/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.63904905, -0.11429453,  0.5860859 ],
        [-0.23696256, -0.6909815 ,  0.22837937],
        [-0.15452969,  0.51794887,  0.78851426],
        [ 0.6990298 , -0.1956085 , -0.5921277 ]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

Sequential model behaves the same. Initially it has no weights.
Only after feeding it with data.

In [12]:
model = keras.Sequential([
    layers.Dense(2, activation='relu'),
    layers.Dense(3, activation='relu'),
    layers.Dense(4)
])

# throws an error
#model.weights

x = tf.ones((1,4))
y = model(x)
model.weights

[<tf.Variable 'dense_7/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.66579866,  0.7089555 ],
        [-0.09803653, -0.5236099 ],
        [ 0.48136687,  0.69046783],
        [ 0.7180288 , -0.12519169]], dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_8/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[ 0.60292363, -0.08458114, -0.31293166],
        [ 0.04722166,  0.7835833 , -0.42419434]], dtype=float32)>,
 <tf.Variable 'dense_8/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_9/kernel:0' shape=(3, 4) dtype=float32, numpy=
 array([[-0.21294701,  0.36732996,  0.5447507 , -0.7668844 ],
        [-0.43822396, -0.25858396,  0.18118453, -0.59841526],
        [-0.2786126 , -0.3452977 ,  0.35734463,  0.39588785]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (1, 2)                    10        
                                                                 
 dense_8 (Dense)             (1, 3)                    9         
                                                                 
 dense_9 (Dense)             (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


You can pass the input in advance (no need to wait for data).
Model needs to know the shape of the data.

In [15]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation='relu'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Input isn't a layer, it is not displayed in the model summary.
Alternatively you can pass input shape in first layer.

In [16]:
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


## Debugging workflow


In [17]:
model = keras.Sequential()
model.add(keras.Input(shape=(250,250,3))) # RGB 250x250x3
model.add(layers.Conv2D(32,5,strides=2,activation='relu'))
model.add(layers.Conv2D(32,3,activation='relu'))
model.add(layers.MaxPooling2D(3))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
